In [1]:
# lfp: spike and rastor
import pynaviz as viz
import pynapple as nap
import numpy as np

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x007\x08\x06\x00\x00\x00\xb6\x1bw\x99\x…

MESA-INTEL: warning: cannot initialize blitter engine
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.
Detected skylake derivative running on mesa i915. Clears to srgb textures will use manual shader clears.


Available devices:
🯄 (default) | Intel(R) Arc(tm) Graphics (MTL) | IntegratedGPU | Vulkan | Mesa 24.0.8-1
❗ | llvmpipe (LLVM 17.0.6, 256 bits) | CPU | Vulkan | Mesa 24.0.8-1 (LLVM 17.0.6)
❗ | Mesa Intel(R) Arc(tm) Graphics (MTL) | IntegratedGPU | OpenGL | 


In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pynapple as nap
from scipy.ndimage import gaussian_filter1d
import pandas as pd

def smooth_series(series, sigma=1):
    smoothed_series = gaussian_filter1d(series, sigma=sigma, mode='constant')
    return pd.Series(smoothed_series, index = series.index)

def smoothAngularTuningCurves(tuning_curves, window = 20, deviation = 3.0):
    new_tuning_curves = {}  
    for i in tuning_curves.columns:
        tcurves = tuning_curves[i]
        offset = np.mean(np.diff(tcurves.index.values))
        padded  = pd.Series(index = np.hstack((tcurves.index.values-(2*np.pi)-offset,
                                                tcurves.index.values,
                                                tcurves.index.values+(2*np.pi)+offset)),
                            data = np.hstack((tcurves.values, tcurves.values, tcurves.values)))
        # smoothed = padded.rolling(window=window,win_type='gaussian',center=True,min_periods=1).mean(std=deviation)
        smoothed = smooth_series(padded, sigma=deviation)
        new_tuning_curves[i] = smoothed.loc[tcurves.index]

    new_tuning_curves = pd.DataFrame.from_dict(new_tuning_curves)

    return new_tuning_curves

def get_memory_map(filepath, nChannels, frequency=20000):
    """Summary
    
    Args:
        filepath (TYPE): Description
        nChannels (TYPE): Description
        frequency (int, optional): Description
    """
    n_channels = int(nChannels)    
    f = open(filepath, 'rb') 
    startoffile = f.seek(0, 0)
    endoffile = f.seek(0, 2)
    bytes_size = 2      
    n_samples = int((endoffile-startoffile)/n_channels/bytes_size)
    duration = n_samples/frequency
    interval = 1/frequency
    f.close()
    fp = np.memmap(filepath, np.int16, 'r', shape = (n_samples, n_channels))        
    timestep = np.arange(0, n_samples)/frequency

    return fp, timestep



# LOADING DATA FROM NWB
data = nap.load_file("/media/caitlin/02B2-43B3/data/Mouse32-140822.nwb")

data



/home/caitlin/venvs/pynaviz/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.8.0 is already loaded.
  return func(args[0], **pargs)
/home/caitlin/venvs/pynaviz/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'core' version 2.4.0 because version 2.7.0 is already loaded.
  return func(args[0], **pargs)
/home/caitlin/venvs/pynaviz/lib/python3.11/site-packages/hdmf/utils.py:668: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.5.0 is already loaded.
  return func(args[0], **pargs)


Mouse32-140822
┍━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┑
│ Keys                  │ Type        │
┝━━━━━━━━━━━━━━━━━━━━━━━┿━━━━━━━━━━━━━┥
│ units                 │ TsGroup     │
│ sws                   │ IntervalSet │
│ rem                   │ IntervalSet │
│ position_time_support │ IntervalSet │
│ epochs                │ IntervalSet │
│ ry                    │ Tsd         │
┕━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━┙

In [3]:
spikes = data["units"]
head_direction = data["ry"]
wake_ep = data["position_time_support"]

# COMPUTING TUNING CURVES
tuning_curves = nap.compute_1d_tuning_curves(
    spikes, head_direction, 120, minmax=(0, 2 * np.pi)
)

# Loading LFP
fp, t = get_memory_map("/media/caitlin/02B2-43B3/data/Mouse32-140822.eeg", 69, 1250)
lfp = nap.TsdFrame(t=t, d=fp)

# The right epoch

ep = nap.IntervalSet(
    start=10717, end=10735
)  # Select an arbitrary interval for plotting

In [4]:
ix = np.array([26, 30, 25, 31, 24, 28, 27, 29, 22, 17, 23, 16, 20, 19, 21, 18, 48, 57, 49, 56, 50, 54, 52, 53])

In [5]:
ix

array([26, 30, 25, 31, 24, 28, 27, 29, 22, 17, 23, 16, 20, 19, 21, 18, 48,
       57, 49, 56, 50, 54, 52, 53])

In [6]:
lfp = lfp.restrict(ep)

In [7]:
lfp = lfp[:, ix]

In [8]:
spikes2 = nap.TsGroup(data= {i: v for i, v in enumerate(spikes.data.values())})

In [9]:
spikes2.set_info(location=spikes.location.values, group=spikes.group.values)

In [10]:
# Smoothing

# COMPUTING TUNING CURVES
tuning_curves = nap.compute_1d_tuning_curves(
    spikes2, head_direction, 120, minmax=(0, 2 * np.pi)
)
tuning_curves = smoothAngularTuningCurves(tuning_curves)

si = nap.compute_1d_mutual_info(tuning_curves, head_direction, wake_ep)

order = np.argsort(np.argmax(tuning_curves.values, 0))

spikes2.set_info(peak=tuning_curves.idxmax(), si=si['SI'])

spikes2 = spikes2.getby_category("location")['adn'].getby_threshold("si", 0.1)

In [11]:
spikes2

Index    rate      location    group    peak     si
-------  --------  ----------  -------  -------  -------
4        0.30207   adn         2        1.59698  1.22464
5        0.87042   adn         2        1.91114  0.16498
6        0.36154   adn         2        1.7017   1.18569
7        10.51737  adn         2        1.64934  1.06815
8        2.62475   adn         2        5.31453  0.49214
9        2.55818   adn         2        2.38237  0.86329
10       7.06715   adn         2        4.05789  0.50904
...      ...       ...         ...      ...      ...
28       1.78011   adn         4        5.99521  0.15728
29       4.23006   adn         4        3.01069  1.83191
30       2.15215   adn         4        4.21497  0.72279
31       0.58829   adn         4        0.70686  1.6722
32       1.12899   adn         4        1.91114  0.93687
33       5.26316   adn         4        3.69137  0.95485
34       1.57122   adn         4        0.86394  1.16114

In [12]:
nw = viz.NeuroWidget(line=[lfp, head_direction],
                     raster=spikes2, 
                     names=["LFP", "Head Direction", "Spikes"],
                     vertical_plots=True,
                     time_interval=ep)

/home/caitlin/repos/fastplotlib/fastplotlib/graphics/_features/_base.py:21: UserWarning: casting float64 array to float32
  warn(f"casting {array.dtype} array to float32")


KeyError: 'Key 0 not in group index.'

In [ ]:
nw.show()

In [14]:
groups = {
 0: [26, 30, 25, 31, 24, 28, 27, 29],
 1: [22, 17, 23, 16, 20, 19, 21, 18],
 2: [48, 57, 49, 56, 50, 54, 52, 53],
 # 3: [58, 51, 60, 55, 61, 59, 62, 63],
 # 4: [45, 36, 41, 34, 37, 35, 33, 32],
 # 5: [39, 46, 38, 47, 40, 44, 43, 42],
 # 6: [15, 8, 14, 9, 13, 10, 12, 11],
 # 7: [7, 0, 6, 1, 5, 2, 4, 3],
 # 8: [64, 65, 66, 67, 68]
}

groups = {
 0: [0, 1, 2, 3, 4, 5, 6, 7],
 1: [8, 9, 10, 11, 12, 13, 14, 15],
 2: [16, 17, 18, 19, 20, 21, 22, 23],
}


In [22]:
nw.figure[0,0].graphics[0][groups[0]].colors = "lime"
nw.figure[0,0].graphics[0][groups[1]].colors = "hotpink"
nw.figure[0,0].graphics[0][groups[2]].colors = "cyan"
# nw.figure[0,0].graphics[0][groups[3]].colors = "orange"
# nw.figure[0,0].graphics[0][groups[4]].colors = "green"
# nw.figure[0,0].graphics[0][groups[5]].colors = "aqua"
# nw.figure[0,0].graphics[0][groups[6]].colors = "magenta"
# nw.figure[0,0].graphics[0][groups[7]].colors = "yellow"
# nw.figure[0,0].graphics[0][groups[8]].colors = "lime"

In [17]:
import matplotlib.pyplot as plt
import numpy as np

def generate_colors(peaks):
    # Normalize peak values to be between 0 and 1
    normalized_peaks = (peaks - np.min(peaks)) / (np.max(peaks) - np.min(peaks))
    
    # Sample colors from the HSV colormap based on normalized peaks
    hsv_colors = plt.cm.hsv(normalized_peaks)
    
    # Ensure the output has the shape (#peaks, 4)
    return hsv_colors

# Example usage
num_lines = len(nw.visuals[2].graphic)  # Replace with the number of lines you have
colors = generate_colors(spikes2.peak.values)

In [18]:
for c, g in zip(colors, nw.visuals[2].graphic):
    g.colors = c

In [23]:
nw.figure.export("/home/caitlin/Desktop/bah.png")